# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [183]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error


In [ ]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [ ]:
spaceship.dropna(inplace=True)
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])
spaceship = spaceship.drop(columns=['Name', 'PassengerId'])
spaceship['VIP'] = spaceship['VIP'].astype('bool')
for col in spaceship.select_dtypes(include=['object']).columns:
    spaceship = pd.get_dummies(spaceship, columns=[col])
spaceship = spaceship.astype('int64')
display(spaceship)

In [ ]:


# Calculate the correlation matrix
corr_matrix = spaceship.corr()

# Filter the correlation matrix for the "Transported" column
corr_matrix_transported = corr_matrix[['Transported']]

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix_transported, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation with Transported')
plt.show()


**Perform Train Test Split**

In [187]:
features = spaceship.drop(columns=['Transported'])
target = spaceship["Transported"]

In [188]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [189]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [ ]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000, n_jobs=-1)

bagging_reg.fit(X_train_norm, y_train)

pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

In [ ]:

pasting_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000, n_jobs=-1,bootstrap=False)

pasting_reg.fit(X_train_norm, y_train)

pred = pasting_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", pasting_reg.score(X_test_norm, y_test))

- Random Forests

In [ ]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)
forest.fit(X_train_norm, y_train)

pred = forest.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", forest.score(X_test_norm, y_test))

- Gradient Boosting

In [ ]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

gb_reg.fit(X_train_norm, y_train)

pred = gb_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test_norm, y_test))

- Adaptive Boosting

In [ ]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

ada_reg.fit(X_train_norm, y_train)

pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test_norm, y_test))

Which model is the best and why?

In [195]:
# The best model is the bagging regressor, sometimes as pasting regressor, with a R2 score of 0.43